### UC Berkeley, MICS, W202-Cryptography
### Week 05 Breakout 3
### Elliptic Curves - Prime Curves - Point Arithmetic - Adding two different points, adding a point to itself, scalar multiplication

Some unique characteristics of non-singular elliptic curves (real or prime):

* if we draw a pure vertical line, it will intersect a point with its reflection.  A point plus its reflection is the point at infinity.  

* if we draw a line between two valid points, the line will intersect a third valid point (and no more).  This concept forms the basis of point addition for adding two different points and yielding a third different point.

* if we draw a tangent line from a single valid point, the tangent line intersects the curve at exactly one other valid point.  This concept forms the basis of point addition for adding a point to itself.

* if we add a point P to itself k times, this forms the basis for scalar multiplication.  For example 5P = P+P+P+P+P. In the next breakout, we will learn that this forms the basis for the Elliptic Curve Discrete Logarithm problem.

We will use an interactive 3rd party web page which demonstrates the concepts of point addition.  Although this is over the real numbers and not over GF(p), it will demonstrate the concept for us.

Note that the slope calculation has division.  We do division in modulo prime p using multiplicative inverses in modulo prime p.  If we are doing it by hand, we can use this relationship:

a^-1 (mod p) = a^(p-2) (mod p)

The SageMath power_mod() function can accept -1 as an exponent.

For point addition for two different points, the slope (lambda) is given by the following formula, very similar to a line in the real number system, but it is done in modulo prime p:

slope

= (y2 - y1) / (x2 - x1) (mod p) 

= (y2 - y1) * (x2 - x1)^-1 (mod p) 

= (y2 - y1) * (x2 - x1)^(p-2) (mod p)

For point addition for a point with itself, the slope (lambda) is given by the 1st derivative of the Weierstrass Equation, but it is done in modulo prime p:

slope

= ((3 * x^2) + A) / (2y) (mod p) 

= ((3 * x^2) + A) * (2y)^-1 (mod p) 

= ((3 * x^2) + A) * (2y)^(p-2) (mod p)

Once we have the slope (lambda), the remainder of the calculation is the same:

x3 = (slope^2 - x1 - x2) (mod p)

y3 = (slope * (x1 - x3)) - y1  (mod p)






#### Use the following (or similar) web based simulator to see how a line between 2 points will intersect a third point, and how a single point will intersect a second point

Be sure and try curves where the ellipse and curve are separate graphs - it still works!

https://www.desmos.com/calculator/ialhd71we3



In [1]:
from sage.all import *

In [2]:
def my_format_plus_minus(x):
    "given an integer, format it with a plus or minus, a blank, and the integer"
    
    if x < 0:
        s = "- " + str(abs(x))
    else:
        s = "+ " + str(abs(x))
    
    return s

In [3]:
def my_add_points(A, B, p, pt_1, pt_2):
    "Given an elliptic curve prime curve defined by A, B, p, and two points, add the two points"
    
    A_string = my_format_plus_minus(A)
    B_string = my_format_plus_minus(B)
    
    x_1 = pt_1[0]
    y_1 = pt_1[1]
    
    x_2 = pt_2[0]
    y_2 = pt_2[1]
    
    print ("\nElliptic Curve Prime Curve:  y^2 = x^3 " + A_string + "x " + B_string + " (mod " + str(p) + ")\n")
    
    E = EllipticCurve(GF(p),[A,B])
    E_pt_3 = E(pt_1) + E(pt_2)
    
    if E_pt_3[2] == 0:
        print ("Points are reflections of each other, sum is the point at infinity\n")
        print ("points: (" + str(x_1) + ", " + str(y_1) + ") + (" + str(x_2) + ", " + str(y_2) + ") = INFINITY")
        return
    
    print ("points: (" + str(x_1) + ", " + str(y_1) + ") + (" + str(x_2) + ", " + str(y_2) + ") = (" \
                + str(E_pt_3[0]) + ", " + str(E_pt_3[1]) + ")\n")
    
    if pt_1 == pt_2:
        print ("Adding the same point to itself:\n")
        print ("    slope")
        print ("       = ((3 * x^2) + A) * (2 * y)^(-1) (mod p)")
        print ("       = ((3 * " + str(x_1) + "^2) + " + str(A) + ") * (2 * " + str(y_1) + ")^(-1) (mod " + str(p) + ")")
        slope = ((3 * power_mod(x_1, 2, p) + A) * power_mod(2 * y_1, -1, p) ) % p
        print ("       = " + str(slope) + "\n")
    else:
        print ("Adding two different points:\n")
        print ("    slope")
        print ("       = (y2 - y1) * (x2 - x1)^(-1) (mod p)")
        print ("       = (" + str(y_2) + " - " + str(y_1) + ") * (" + str(x_2) + " - " \
                + str(x_1) + ")^(-1) (mod " + str(p) + ")")
        slope = ((y_2 - y_1) * power_mod(x_2 - x_1, -1, p) ) %p
        print ("       = " + str(slope) + "\n")
    
    x_3 = (slope ** 2 - x_1 - x_2) % p
    
    print ("    x3 = (slope^2 - x1 - x2) (mod p)")
    print ("       = (" + str(slope) + "^2 - " + str(x_1) + " - " + str(x_2) + ") (mod " + str(p) + ")")
    print ("       = " + str(x_3) + "\n")
    
    y_3 = (slope * (x_1 - x_3) - y_1) % p
    
    print ("    y3 = (slope * (x1 - x3)) - y1 (mod p)")
    print ("       = (" + str(slope) + " * (" + str(x_1) + " - " + str(x_3) + ")) - " \
                    + str(y_1) + " (mod " + str(p) + ")")
    print ("       = " + str(y_3) + "\n")
    
    

#### Examples of adding a point to its reflection which yields the point at infinity

In [4]:
# in this example, we use a point and its reflection
# remember -1 (mod 11) = 10, so (6,1) and (6,10) are reflections

A = 4
B = 3
p = 11

pt_1 = (6,1)
pt_2 = (6,10)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 11)

Points are reflections of each other, sum is the point at infinity

points: (6, 1) + (6, 10) = INFINITY


In [5]:
# in this example, we use a point and its reflection
# remember -4 (mod 13) = 9, so (10,4) and (10,9) are reflections

A = 4
B = 3
p = 13

pt_1 = (10,4)
pt_2 = (10,9)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 13)

Points are reflections of each other, sum is the point at infinity

points: (10, 4) + (10, 9) = INFINITY


In [6]:
# in this example, we use a point and its reflection
# remember -29 (mod 97) = 68, so (51,29) and (51,68) are reflections

A = 4
B = 3
p = 97

pt_1 = (51,29)
pt_2 = (51,68)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 97)

Points are reflections of each other, sum is the point at infinity

points: (51, 29) + (51, 68) = INFINITY


#### Examples of adding two different points

In [7]:
A = 4
B = 3
p = 11

pt_1 = (3,8)
pt_2 = (5,4)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 11)

points: (3, 8) + (5, 4) = (7, 0)

Adding two different points:

    slope
       = (y2 - y1) * (x2 - x1)^(-1) (mod p)
       = (4 - 8) * (5 - 3)^(-1) (mod 11)
       = 9

    x3 = (slope^2 - x1 - x2) (mod p)
       = (9^2 - 3 - 5) (mod 11)
       = 7

    y3 = (slope * (x1 - x3)) - y1 (mod p)
       = (9 * (3 - 7)) - 8 (mod 11)
       = 0



In [8]:
A = 4
B = 3
p = 13

pt_1 = (3,4)
pt_2 = (7,6)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 13)

points: (3, 4) + (7, 6) = (0, 4)

Adding two different points:

    slope
       = (y2 - y1) * (x2 - x1)^(-1) (mod p)
       = (6 - 4) * (7 - 3)^(-1) (mod 13)
       = 7

    x3 = (slope^2 - x1 - x2) (mod p)
       = (7^2 - 3 - 7) (mod 13)
       = 0

    y3 = (slope * (x1 - x3)) - y1 (mod p)
       = (7 * (3 - 0)) - 4 (mod 13)
       = 4



In [9]:
A = 4
B = 3
p = 53

pt_1 = (8,21)
pt_2 = (42,18)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 53)

points: (8, 21) + (42, 18) = (18, 36)

Adding two different points:

    slope
       = (y2 - y1) * (x2 - x1)^(-1) (mod p)
       = (18 - 21) * (42 - 8)^(-1) (mod 53)
       = 42

    x3 = (slope^2 - x1 - x2) (mod p)
       = (42^2 - 8 - 42) (mod 53)
       = 18

    y3 = (slope * (x1 - x3)) - y1 (mod p)
       = (42 * (8 - 18)) - 21 (mod 53)
       = 36



In [10]:
A = 4
B = 3
p = 97

pt_1 = (21,6)
pt_2 = (64,55)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 97)

points: (21, 6) + (64, 55) = (8, 81)

Adding two different points:

    slope
       = (y2 - y1) * (x2 - x1)^(-1) (mod p)
       = (55 - 6) * (64 - 21)^(-1) (mod 97)
       = 44

    x3 = (slope^2 - x1 - x2) (mod p)
       = (44^2 - 21 - 64) (mod 97)
       = 8

    y3 = (slope * (x1 - x3)) - y1 (mod p)
       = (44 * (21 - 8)) - 6 (mod 97)
       = 81



#### Examples of adding a point to itself

In [11]:
A = 4
B = 3
p = 11

pt_1 = (5,7)
pt_2 = (5,7)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 11)

points: (5, 7) + (5, 7) = (10, 8)

Adding the same point to itself:

    slope
       = ((3 * x^2) + A) * (2 * y)^(-1) (mod p)
       = ((3 * 5^2) + 4) * (2 * 7)^(-1) (mod 11)
       = 8

    x3 = (slope^2 - x1 - x2) (mod p)
       = (8^2 - 5 - 5) (mod 11)
       = 10

    y3 = (slope * (x1 - x3)) - y1 (mod p)
       = (8 * (5 - 10)) - 7 (mod 11)
       = 8



In [12]:
A = 4
B = 3
p = 13

pt_1 = (8,1)
pt_2 = (8,1)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 13)

points: (8, 1) + (8, 1) = (7, 6)

Adding the same point to itself:

    slope
       = ((3 * x^2) + A) * (2 * y)^(-1) (mod p)
       = ((3 * 8^2) + 4) * (2 * 1)^(-1) (mod 13)
       = 7

    x3 = (slope^2 - x1 - x2) (mod p)
       = (7^2 - 8 - 8) (mod 13)
       = 7

    y3 = (slope * (x1 - x3)) - y1 (mod p)
       = (7 * (8 - 7)) - 1 (mod 13)
       = 6



In [13]:
A = 4
B = 3
p = 13

pt_1 = (8,1)
pt_2 = (8,1)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 13)

points: (8, 1) + (8, 1) = (7, 6)

Adding the same point to itself:

    slope
       = ((3 * x^2) + A) * (2 * y)^(-1) (mod p)
       = ((3 * 8^2) + 4) * (2 * 1)^(-1) (mod 13)
       = 7

    x3 = (slope^2 - x1 - x2) (mod p)
       = (7^2 - 8 - 8) (mod 13)
       = 7

    y3 = (slope * (x1 - x3)) - y1 (mod p)
       = (7 * (8 - 7)) - 1 (mod 13)
       = 6



In [14]:
A = 3
B = 8
p = 13

pt_1 = (2,3)
pt_2 = (2,3)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 3x + 8 (mod 13)

points: (2, 3) + (2, 3) = (12, 11)

Adding the same point to itself:

    slope
       = ((3 * x^2) + A) * (2 * y)^(-1) (mod p)
       = ((3 * 2^2) + 3) * (2 * 3)^(-1) (mod 13)
       = 9

    x3 = (slope^2 - x1 - x2) (mod p)
       = (9^2 - 2 - 2) (mod 13)
       = 12

    y3 = (slope * (x1 - x3)) - y1 (mod p)
       = (9 * (2 - 12)) - 3 (mod 13)
       = 11



In [15]:
A = 4
B = 3
p = 97

pt_1 = (21,6)
pt_2 = (21,6)

my_add_points(A, B, p, pt_1, pt_2)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 97)

points: (21, 6) + (21, 6) = (61, 65)

Adding the same point to itself:

    slope
       = ((3 * x^2) + A) * (2 * y)^(-1) (mod p)
       = ((3 * 21^2) + 4) * (2 * 6)^(-1) (mod 97)
       = 54

    x3 = (slope^2 - x1 - x2) (mod p)
       = (54^2 - 21 - 21) (mod 97)
       = 61

    y3 = (slope * (x1 - x3)) - y1 (mod p)
       = (54 * (21 - 61)) - 6 (mod 97)
       = 65



#### Examples of scalar multiplication

Notice the patterns that repeat

In [16]:
def my_scalar_multiplication(A, B, p, pt, k):
    "Given an elliptic curve prime curve defined by A, B, p, and a point and an integer, perform scalar multiplication"
    
    A_string = my_format_plus_minus(A)
    B_string = my_format_plus_minus(B)
        
    print ("\nElliptic Curve Prime Curve:  y^2 = x^3 " + A_string + "x " + B_string + " (mod " + str(p) + ")\n")
    
    E = EllipticCurve(GF(p),[A,B])
    
    E_pt = E(pt)
    
    for i in range(1, k+1):
        
        E_s = E_pt * i
        
        if E_s[2] == 0:
            s = "INFINITY"
        else:
            s = "(" + str(E_s[0]) + ", " + str(E_s[1]) + ")"
        
        print (str(i) + " * (" + str(E_pt[0]) + ", " + str(E_pt[1]) + ") = " + s )
    

In [17]:
A = 4
B = 3
p = 11

pt = (5,4)

k = 100

my_scalar_multiplication(A, B, p, pt, k)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 11)

1 * (5, 4) = (5, 4)
2 * (5, 4) = (10, 3)
3 * (5, 4) = (0, 6)
4 * (5, 4) = (0, 5)
5 * (5, 4) = (10, 8)
6 * (5, 4) = (5, 7)
7 * (5, 4) = INFINITY
8 * (5, 4) = (5, 4)
9 * (5, 4) = (10, 3)
10 * (5, 4) = (0, 6)
11 * (5, 4) = (0, 5)
12 * (5, 4) = (10, 8)
13 * (5, 4) = (5, 7)
14 * (5, 4) = INFINITY
15 * (5, 4) = (5, 4)
16 * (5, 4) = (10, 3)
17 * (5, 4) = (0, 6)
18 * (5, 4) = (0, 5)
19 * (5, 4) = (10, 8)
20 * (5, 4) = (5, 7)
21 * (5, 4) = INFINITY
22 * (5, 4) = (5, 4)
23 * (5, 4) = (10, 3)
24 * (5, 4) = (0, 6)
25 * (5, 4) = (0, 5)
26 * (5, 4) = (10, 8)
27 * (5, 4) = (5, 7)
28 * (5, 4) = INFINITY
29 * (5, 4) = (5, 4)
30 * (5, 4) = (10, 3)
31 * (5, 4) = (0, 6)
32 * (5, 4) = (0, 5)
33 * (5, 4) = (10, 8)
34 * (5, 4) = (5, 7)
35 * (5, 4) = INFINITY
36 * (5, 4) = (5, 4)
37 * (5, 4) = (10, 3)
38 * (5, 4) = (0, 6)
39 * (5, 4) = (0, 5)
40 * (5, 4) = (10, 8)
41 * (5, 4) = (5, 7)
42 * (5, 4) = INFINITY
43 * (5, 4) = (5, 4)
44 * (5, 4) = (10, 3)
4

In [18]:
A = 4
B = 3
p = 53

pt = (42,18)

k = 100

my_scalar_multiplication(A, B, p, pt, k)


Elliptic Curve Prime Curve:  y^2 = x^3 + 4x + 3 (mod 53)

1 * (42, 18) = (42, 18)
2 * (42, 18) = (5, 25)
3 * (42, 18) = (30, 27)
4 * (42, 18) = (18, 17)
5 * (42, 18) = (8, 32)
6 * (42, 18) = (47, 9)
7 * (42, 18) = (16, 20)
8 * (42, 18) = (11, 0)
9 * (42, 18) = (16, 33)
10 * (42, 18) = (47, 44)
11 * (42, 18) = (8, 21)
12 * (42, 18) = (18, 36)
13 * (42, 18) = (30, 26)
14 * (42, 18) = (5, 28)
15 * (42, 18) = (42, 35)
16 * (42, 18) = INFINITY
17 * (42, 18) = (42, 18)
18 * (42, 18) = (5, 25)
19 * (42, 18) = (30, 27)
20 * (42, 18) = (18, 17)
21 * (42, 18) = (8, 32)
22 * (42, 18) = (47, 9)
23 * (42, 18) = (16, 20)
24 * (42, 18) = (11, 0)
25 * (42, 18) = (16, 33)
26 * (42, 18) = (47, 44)
27 * (42, 18) = (8, 21)
28 * (42, 18) = (18, 36)
29 * (42, 18) = (30, 26)
30 * (42, 18) = (5, 28)
31 * (42, 18) = (42, 35)
32 * (42, 18) = INFINITY
33 * (42, 18) = (42, 18)
34 * (42, 18) = (5, 25)
35 * (42, 18) = (30, 27)
36 * (42, 18) = (18, 17)
37 * (42, 18) = (8, 32)
38 * (42, 18) = (47, 9)
39 * (42, 18) =